In [87]:
#Scoring by testing various permutations of individual fields
#thought process is the following
#Actor 1 will pull GT7,GT6,GT10,GT16 then 'empty'
#Actor 2 will pull GT7,GT6,GT10,GT16 then 'empty'
#Actor1mention  will pull GT7,GT6,GT10,GT16 then 'empty'
#Actor2mention  will pull GT7,GT6,GT10,GT16 then 'empty'
#Geo we will switch up to try GT10 GT16, GT7, GT6
#Cameo will be GT7,GT16, GT10, GT6
#clustering across cameo codes we need to be careful as to not cross cameos from different types or do we???
#
# 1.) read in all clusters from tamr
# 2.) find how many clusters we have
# 2a.) get distinct clusters
# 3.) start processing each cluster
#   3a.) read in custer[x]
#   3b.) set fields based on rules from above (how we do this)
#   3c.) print out record and where each vaue came from (how do we keep track of where everything came from)
#
#   3d.) score the record
#   3e.) get next cluster

# for i,facts in enumerator(clusters):
#  if facts[i][8] == 'gt7' && facts[i][8] not empty:
#    actor1 = facts[i][2]  <--- get actor1 from gt7 record
# otherwise we would have to check how many gt7 records exist in cluster and check each of those before moving on
import sys
import os
from munkres import Munkres
import numpy as np
import re
import csv
import itertools

import collections

def reduce_vendordata (vendorlist,actor1_source,actor2_source,actor1mention_source,actor2mention_source,cameocode_source,geofullname_source):
    
    #print len(vendorlist)
    #print vendorlist

    while_counter = 0
    vendoractor1 = []
    vendoractor2 = []
    vendoractor1mentions = []
    vendoractor2mentions = []
    vendorfullgeonames = []
    vendorcameocodes = []
    vendorguid = []
    vendorentityid = []
    vendorcluster = []
    vendorglobaleventid = []
    
    

    while while_counter < len(vendorlist):
        #get all of our values for scoring
    
        #print while_counter,dsvendor[while_counter][1]
        #vendoreventid.append(vendorlist[while_counter][65])
        if actor1_source == vendorlist[while_counter][40]:
            vendoractor1.append(vendorlist[while_counter][18])
            
        if actor2_source == vendorlist[while_counter][40]:
            vendoractor2.append(vendorlist[while_counter][34])
        if actor1mention_source == vendorlist[while_counter][40]:
            vendoractor1mentions.append(vendorlist[while_counter][17])
        if actor2mention_source == vendorlist[while_counter][40]:
            vendoractor2mentions.append(vendorlist[while_counter][33])
        if geofullname_source == vendorlist[while_counter][40]:
            #print 'geo loop check',vendorlist[while_counter][3]
            vendorfullgeonames.append(vendorlist[while_counter][3])
        if cameocode_source == vendorlist[while_counter][40]:
            vendorcameocodes.append(vendorlist[while_counter][60])
        vendorguid.append(vendorlist[while_counter][78])
        vendorcluster.append(vendorlist[while_counter][93])
        vendorentityid.append(vendorlist[while_counter][0])
        vendorglobaleventid.append(vendorlist[while_counter][65])
        
        #clusterid2 = vendorlist[while_counter][8]
        
        #print 'in flatten clusterid',clusterid2
        while_counter += 1
    

    #lets check our values
    #list(set(gt_guid))
    #filter(None, lstr)
    #we are removing all null entries and duplicates just to get to the bare minimums
    vendoractor1 = filter(None,list(set(vendoractor1)))
    vendoractor2 = filter(None,list(set(vendoractor2)))
    vendoractor1mentions = filter(None,list(set(vendoractor1mentions)))
    vendoractor2mentions = filter(None,list(set(vendoractor2mentions)))
    vendorfullgeonames = filter(None,list(set(vendorfullgeonames)))
    vendorcameocodes = filter(None,list(set(vendorcameocodes)))
    vendorguid = filter(None,list(set(vendorguid)))

    #if vendorguid[0] == '6b4f7c60-1141-4259-b551-15ebb0b88b84':
        #print 'vendorlist',vendorlist
        #print 'a1',vendoractor1
        #print 'a2',vendoractor2
 
    
    '''
    print 'a1',vendoractor1
    print 'a2',vendoractor2
    print 'a1m',vendoractor1mentions
    print 'a2m',vendoractor2mentions
    print 'fgn',vendorfullgeonames
    print 'cameo',vendorcameocodes
    print 'guid',vendorguid
    #print 'id',vendorid
    #print clusterid
    '''
    
    return vendoractor1,vendoractor2,vendoractor1mentions,vendoractor2mentions,vendorfullgeonames,vendorcameocodes,vendorguid,vendorentityid,vendorglobaleventid
    #end of vendor data function



class Vendor_Data(object):
    def __init__(self,data,pointer):
        #self.globaleventid = data[pointer][0]
        
        #print 'in vendor event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][0]
        self.actor2name = data[pointer][1]
        self.actor1mention = data[pointer][2]
        self.actor2mention = data[pointer][3]
        self.actiongeo_fullname = data[pointer][4]
        self.eventcode = data[pointer][5]
        self.guid = data[pointer][6]
        #self.cluster = data[pointer][7]
        
class GroundTruth_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in ground truth event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]

def compute_score(true_event, test_event):
    
	#print 'in compute_score check our gt values',true_event.actor1name
	#print 'in compute_score check our vendor values',test_event.actor1name
	# """
	# Evaluate the score of test event
	# parameters:
	#    true_event: json 
	#    test_event: json
	# """
	################
	# score actor1 #
	################
	# only evaluate the actor1name with true event using partial match
	actor1_score = 0.0
	test_actor1 = test_event.actor1name
	#print 'print vendor actor1',test_actor1    
	# if there are multiple sources, we match the actor1 with 
	# highest score. In Reciproal case, we compare the actor1/2 with 
	# the same set of Reciprocal texts.
	if true_event.actor1name:
		sources = true_event.actor1name.split(";")
	else:
		sources = []
	if true_event.actor1mention:
		new_sources = true_event.actor1mention.split(";")
		for new_source in new_sources:
			sources.append(new_source)
	#print 'here'
	act_score = 0.
	for source in sources:
		act_score = 3.0 if text_partial_match(source, test_actor1) else 0.
		if act_score > actor1_score:
			actor1_score = act_score
	#print 'actor1 score',act_score
	################
	# score actor2 #
	################
	actor2_score = 0.0
	test_actor2 = test_event.actor2name
	#print 'print vendor actor2',test_actor2
    
	if true_event.actor2name:
		targets = true_event.actor2name.split(";")
	else:
		targets = []
	if true_event.actor2mention:
		new_targets = true_event.actor2mention.split(";")
		for new_target in new_targets:
			targets.append(new_target)

	for target in targets:
		#print 'in targets',target    
		actor_score = 3.0 if text_partial_match(target, test_actor2) else 0
		#print 'actor2 score',actor_score
		if act_score > actor2_score:
			actor2_score = act_score

	##################
	# Score Location #
	##################
	# Match Location with partial match
	location_score = 0.0
	true_location = true_event.actiongeo_fullname.split(";")
	test_location = test_event.actiongeo_fullname.strip()
	#print 'print gt location',true_location    
	#print 'print vendor location',test_location
	if len(true_location) == 0:
		if test_location == "":
			location_score = 3.0
		else:
			location_score = 0.0
	else:
		for loc in true_location:
			tem_score = 3.0 if text_partial_match(loc, test_location) else 0.0
			if tem_score > location_score:
				location_score = tem_score
    
	################
	# score action #
	################
	# compute the action score with two level: root and subtype
	action_score = 0.0
	#print 'event code check',test_event.eventcode    
	true_code = true_event.eventcode.split(" ")[0]
	test_code = test_event.eventcode.split(" ")[0]
    
	#print 'checking our event code values for gt -->',true_code
	#print 'checking our test code values for gt -->',test_code
    
	if test_code is None or len(test_code) < 2:
		#print 'in action score fail'        
		action_score = 0.0
	else:
		if true_code[:2] == test_code[:2]:
			#print 'checking action score',true_code[:2],test_code[:2],action_score
			action_score += 2.0
			if len(test_code) > 3 and test_code[2] == true_code[2]:
				action_score += 1.0
    
	###############
	# Date Score ##
	###############
	# since the Date field in the GSR is not normalized, we don't compute the
	# score here
	date_score = 0.0

	final_score = [actor1_score, actor2_score, location_score, action_score, date_score]

    
	fields = ["Actor1", "Actor2", "Location", "Action", "Date"]
	#print 'Final_Score -->',sum(final_score),dict(zip(fields, final_score))

	return sum(final_score), dict(zip(fields, final_score))

def text_partial_match(true_text, test_text):
	"""
	partial match the true_text and test_text based on following step
	1> transform the text to lowercase 
	2> split the text by space or comma and ignore ''
	3> if any pair of elements from two different text arraies equal to each
	other, return True else False
	"""
	true_text_splits = set([r for r in re.split(r" |,", true_text.lower()) if r != ''])
	test_text_splits = set([r for r in re.split(r" |,", test_text.lower()) if r != ''])
    
	if true_text_splits.intersection(test_text_splits):
		return True
	else:
		return False
        
        
        
def global_match(ground_truth, vendor_events):
    """
    matching the true events with test events with global optimal score
    """
    
    gt_guid = []    
    vendor_guid = []
    vendor_score = 0.00
    
    for records, true_event in enumerate(ground_truth):
        #if records == len(ground_truth): break        
        #extract guid

        #print records        
        gt_guid.append(ground_truth[records][7])

    #print 'vendor_events',vendor_events
    for counter, records in enumerate(vendor_events):
        #extract guid
        #print records
        vendor_guid.append(vendor_events[counter][6])
    
    gtguids = list(set(gt_guid))

    #print 'here is my gt set', len(gtguids)
    
    vendorguid = list(set(vendor_guid))
    #print 'here is my vendor set', len(vendorguid)
    
    #let's check how many values are in our vendor set
    guids_to_process = []    
    for guids,groundtruth in enumerate(gtguids):
        #print 'checking',gt         
        if gtguids[guids] in vendorguid:
            guids_to_process.append(gtguids[guids])  
    #print 'here are my guids_to_process',guids_to_process 

    #ok, so now lets iterate through out guids
    #print 'before guid count'
    #open our file to write our results out to

    #score_matrix = []        
    for processing in guids_to_process:
        #print 'processing guids'
        vendor_1 = filter(lambda vendor_events: vendor_events[6] in processing, vendor_events) 
        gt_1  = filter(lambda ground_truth: ground_truth[7] in processing, ground_truth)        
        
        
        score_matrix = np.zeros((len(gt_1),len(vendor_1)))  
        #print 'vendor length',vendor_1
        #print 'gt length',gt_1
        
        score_detail = {}
        for i, true_event in enumerate(gt_1):
            #print 'loop counter',i
            true_events = GroundTruth_Data(gt_1,i)

            for j, test_event in enumerate(vendor_1):   
    
                test_events = Vendor_Data(vendor_1,j)    
                
                score, score_list = compute_score(true_events, test_events)
                #print 'score on return',score                
                if score > 0:
                    #print 'here is the score for guid:',processing,score            
                    score_matrix[i, j] = score
                    score_detail[(i,j)] = {"score": score, "detail": score_list, "pair": [test_event]} 
                if processing == '0070cb46-4a4e-4964-8904-6283bd1c34e0' and score >0:
                    print 'processing',processing,score,score_list,test_event 
                    
                    
                
        cost_matrix = 10000 - score_matrix
        m = Munkres()

        #print 'after munkres'
        indexes = m.compute(cost_matrix.tolist())
        #print 'here are my indexes',indexes
        total_score = sum(score_matrix[zip(*indexes)])
        #print 'total_score',total_score
        event_pairs = []
        #going to track total score 
        #print 'the total score field is',total_score
        vendor_score += total_score        
        print 'The total score for guid %s is: %s'%(processing,total_score) 
        unmatched_events = []
        spurious_events = []
        #matched_true_ids, matched_test_ids = zip(*indexes)
        #print matched_true_ids
        
        #writer = csv.writer(open("/Users/Pete/clusterGT10.csv",'a'))
        #writer.writerow([test_event])
        #if processing =='6b4f7c60-1141-4259-b551-15ebb0b88b84':
        #print 'processing our test guid',processing
        #print 'score',vendor_score
           

        
    return vendor_score     


# function to build our record to score
def buildrecord(tamr_records,actor1_source,actor2_source,actor1mention_source,actor2mention_source,cameocode_source,geofullname_source):

    actor1_from = actor1_source
    actor2_from = actor2_source
    actor1mention_from = actor1mention_source
    actor2mention_from = actor2mention_source
    cameocode_from = cameocode_source
    geofullname_from = geofullname_source
    vendor_matrix = None
    
    #print 'tamr_records in buildrecord',tamr_records   
    #check how many instances of each vendor type we have
    #gt7_count = sum(x.count('gt7') for x in tamr_records)
    #gt16_count = sum(x.count('aic.gt16') for x in tamr_records)
    #gt10_count = sum(x.count('gt10') for x in tamr_records)
    #gt6_count = sum(x.count('gt6') for x in tamr_records)
    
    


    #print 'gt7',gt7_count
    #print 'gt16',gt16_count
    #print 'gt10',gt10_count
    #print 'gt6',gt6_count
    
    #print 'actor1source',actor1_source
    #print 'actor2source',actor2_source
    #print 'geofull',geofullname_source
    
    #print 'len of tamr_records',len(tamr_records)
    
    vendoractor1,vendoractor2,vendoractor1mentions,vendoractor2mentions,vendorfullgeonames,vendorcameocodes,vendorguids,vendorentityid,vendorglobaleventid = reduce_vendordata(tamr_records,actor1_source,actor2_source,actor1mention_source,actor2mention_source,cameocode_source,geofullname_source)

    if not vendoractor1:
        vendoractor1.append('EMPTY')
    if not vendoractor2:
        vendoractor2.append('EMPTY')
    if not vendoractor1mentions:
        vendoractor1mentions.append('EMPTY')
    if not vendoractor2mentions:
        vendoractor2mentions.append('EMPTY')
    if not vendorfullgeonames:
        vendorfullgeonames.append('EMPTY')
    if not vendorcameocodes:
        vendorcameocodes.append('EMPTY')
    if not vendorguids:
        vendorguids.append('EMPTY')
    #print 'before matrix build'
    #if vendoractor1[0]=='EMPTY'and vendoractor2[0]=='EMPTY'and vendoractor1mentions == 'EMPTY'and vendoractor2mentions =='EMPTY' and vendorcameocodes == 'EMPTY':
    vendor_matrix = list(itertools.product(vendoractor1,vendoractor2,vendoractor1mentions,vendoractor2mentions,vendorfullgeonames,vendorcameocodes,vendorguids))
        #print vendor_matrix
    #if vendorguids[0]== '6b4f7c60-1141-4259-b551-15ebb0b88b84':
        #print vendor_matrix
    #writer = csv.writer(open("/Users/Pete/July25FlattenclusterGT7.csv",'a'))
    #writer.writerow([vendor_matrix])    
      
    #record_to_score = []
    #record_to_score.extend([actor1pulled,actor2pulled,actor1mentionpulled,actor2mentionpulled,cameocodepulled,geofullnamepulled,guidpulled,actor1_from,actor2_from,actor1mention_from,actor2mention_from,cameocode_from,geofullname_from])
    
    #print 'record to score',vendor_matrix
    
    #print actor1,actor2,actor1mention,actor2mention,cameocode,geofullname,actor1_from,actor2_from,actor1mention_from,actor2mention_from,cameocode_from,geofullname_from
     
    #return str(record_to_score).replace('[','').replace(']','')
    return vendor_matrix


#****************
#main body of program
#****************


actor1_processing = raw_input('Which Actor1 vendor should we score on? ')
actor2_processing = raw_input('Which Actor2 vendor should we score on? ')
actor1mention_processing = raw_input('Which Actor1Mention vendor should we score on? ')
actor2mention_processing = raw_input('Which Actor2 Mention vendor should we score on? ')
cameocode_processing = raw_input('Which Cameo code vendor should we score on? ')
geofullname_processing = raw_input('Which GeoFullName vendor should we score on? ')
cluster_total_score = 0.00
vendorscore=0.00

#read in tamr data
#tamr_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gfcluster1.score.csv'
#tamr_data = '/Users/Pete/Customers/2016.Feb Greenfield/threefulltamrrecords.csv'
#tamr_data = '/Users/Pete/Customers/2016.Feb Greenfield/onefulltamrrecord.csv'
#tamr_data = '/Users/Pete/july23tamrclusters.csv'
tamr_data ='/Users/Pete/tamrclustersJuly26run1.csv'
#tamr_data = '/Users/Pete/Customers/OneGuidGT7.csv'

with open(tamr_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    tamrds = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists

#read in ground truth data
#gt_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gfgroundtruthtest1.score.csv'
gt_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/groundtruthcombined.score.csv'
#gt_data = '/Users/Pete/Customers/gt.matchtest1'

with open(gt_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    dsgold = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists

#print len(tamrds),tamrds
#print len(dsgold),dsgold

clusters_to_process = []
a=[]
#get all the clusters
for counter,clusters in enumerate(tamrds):
    clusters_to_process.append(tamrds[counter][93])
    
clusters = list(set(clusters_to_process))    
#print 'our unique cluster ids',len(clusters)

#let's loop through and filter our data we don't need right now

#writer = open('/Users/Pete/July25FlattenclusterGT7e.csv','a')
#out = open('/Users/Pete/Customers/2016.April SpendDifference/distributorMapped.csv','a')
#wr = csv.writer(writer, delimiter = ',')
#wr = csv.writer(writer, dialect='excel')

for count,elements in enumerate(clusters):
    
    #if count % 500 == 0:
        #print 'processing on count',count
    dstamr_clusters = filter(lambda tamrds: tamrds[93] in elements,tamrds)
    
# for i,facts in enumerator(clusters):
#  if facts[i][8] == 'gt7' && facts[i][8] not empty:
#    actor1 = facts[i][2]  <--- get actor1 from gt7 record
# otherwise we would have to check how many gt7 records exist in cluster and check each of those before moving on

    #call function to build record to score from our cluster
    tamr_record_to_score = buildrecord(dstamr_clusters,actor1_processing,actor2_processing,actor1mention_processing,actor2mention_processing,cameocode_processing,geofullname_processing)    
    if tamr_record_to_score != None:
        a.extend(tamr_record_to_score)
#print 'record to score',a
#print 'works',tamr_record_to_score
#print "length of a",a
#print 'after matrix build'
#writer.writerow(a) 
    #wr.writerow(tamr_record_to_score)
print 'moving onto scoring'
vendorscore = global_match(dsgold,a)

#vendorscore +=cluster_total_score
       
#if cluster_total_score > 0:
    #print 'using input from sources %s the score for is %s'%(actor1_processing,cluster_total_score)
    #print cluster_total_score
#cluster_total_score = cluster_total_score + cluster_total_score
    
print 'Total for this vendor',vendorscore

print 'fini'
    

Which Actor1 vendor should we score on? GT14
Which Actor2 vendor should we score on? GT14
Which Actor1Mention vendor should we score on? GT14
Which Actor2 Mention vendor should we score on? GT14
Which Cameo code vendor should we score on? GT14
Which GeoFullName vendor should we score on? GT14
moving onto scoring
The total score for guid e881cff8-6b30-4b1e-8b9a-3bce1abab803 is: 0.0
The total score for guid 825b189d-9f18-4fd4-8cdc-a698afb14c69 is: 0.0
The total score for guid ccda9cc8-bd9b-4317-978c-1582f41d3498 is: 0.0
The total score for guid b54bbfe8-1b16-45e1-8802-4e1e6765f8f8 is: 0.0
The total score for guid 7e1e8bc4-f58f-4043-87d2-81b5a4293e67 is: 0.0
The total score for guid 1ad37b32-6521-4c88-aa87-08f081e2bb6b is: 0.0
The total score for guid 2a0e4998-d1ce-444b-aba2-d5bf6e6a7d09 is: 0.0
The total score for guid 82af1c43-9573-44d6-8a28-296f481a6006 is: 0.0
The total score for guid 3ba7e8be-b731-43bd-86d8-b145ad0fdfc1 is: 0.0
The total score for guid 49643d38-5018-4e65-b4ff-fcd78ac